In [ ]:
!pip install datasets
!pip install transformers
!pip install tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors

In [ ]:
from transformers import PreTrainedTokenizerFast

In [ ]:
# Load AG_NEWS dataset using the datasets library
dataset = load_dataset("ag_news")
dataset_train = dataset["train"]
dataset_test = dataset["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
len(dataset_train)

120000

In [ ]:
# Initialize a BPE tokenizer
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

# Set the pre-tokenizer to split on whitespace
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Set up the trainer with special tokens
trainer = trainers.BpeTrainer(
    vocab_size=1000,
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

# Define a generator to yield batches of text
def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset_train), batch_size):
        yield dataset_train[i:i + batch_size]["text"]

# Train the tokenizer
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

# Set the post-processor to add special tokens
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)
# Enable truncation and padding
tokenizer.enable_truncation(max_length=128)
tokenizer.enable_padding(pad_id=tokenizer.token_to_id("[PAD]"), pad_token="[PAD]", length=128)

In [ ]:
tokenizer.save("custom_tokenizer.json")

In [ ]:
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")

In [ ]:
def preprocess_data(examples):
    # Tokenize the text
    encodings = tokenizer.encode_batch(examples["text"])

    # Extract input IDs and attention masks
    input_ids = [encoding.ids for encoding in encodings]
    attention_masks = [encoding.attention_mask for encoding in encodings]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "label": examples["label"]
    }


In [ ]:
# Apply preprocessing
ds_train = dataset_train.map(preprocess_data, batched=True)
ds_test = dataset_test.map(preprocess_data, batched=True)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
ds_train[0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'input_ids': [1,
  54,
  150,
  202,
  16,
  33,
  996,
  34,
  699,
  33,
  215,
  157,
  99,
  95,
  923,
  215,
  11,
  219,
  12,
  219,
  15,
  383,
  692,
  15,
  78,
  245,
  133,
  14,
  54,
  150,
  202,
  104,
  124,
  10,
  78,
  63,
  82,
  286,
  71,
  101,
  58,
  61,
  112,
  103,
  745,
  144,
  15,
  62,
  84,
  73,
  605,
  14,
  192,
  151,
  64,
  101,
  66,
  104,
  96,
  362,
  16,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [ ]:
# Define Basic Embedding Model
class BasicEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        embedded = torch.mean(embedded, dim=1)  # Averaging embeddings
        logits = self.fc(embedded)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

In [ ]:
# Model setup
embed_dim = 100
num_classes = 4
vocab_size = 1000
model = BasicEmbeddingModel(vocab_size, embed_dim, num_classes)
# Show model summary
print(model)

BasicEmbeddingModel(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


In [ ]:
# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

In [ ]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
)

In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    processing_class=fast_tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Train model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: jdmartinev (jdmartinev_) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.483800,0.478723,0.831579,0.831190
2,0.432400,0.443806,0.845658,0.845302
3,0.425900,0.438144,0.848158,0.847845


TrainOutput(global_step=11250, training_loss=0.5131447672526042, metrics={'train_runtime': 111.5174, 'train_samples_per_second': 3228.195, 'train_steps_per_second': 100.881, 'total_flos': 0.0, 'train_loss': 0.5131447672526042, 'epoch': 3.0})